In [1]:
import re
import pandas as pd
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
from bs4 import BeautifulSoup
import json

In [2]:
salary_ranges = ['0-70000','70000-120000','120000-999999']
# declare an empty list for storing urls
job_urls = {}

for salary_range in salary_ranges:
    job_urls[salary_range]=[]
    for page in range(1,50):
        # iteraterate through the 50 pages of jobs
        r = requests.get("https://www.seek.com.au/data-analyst-jobs?page={0}&salaryrange={1}&salarytype=annual".format(str(page),salary_range))

        # turn into a BeautifulSoup object
        soup = BeautifulSoup(r.text, 'lxml')

        # find urls on page
        urls = soup.find_all(attrs={'data-automation':"jobTitle"})
        
        if urls != []: 
            job_urls[salary_range].extend(urls)

If there is one job which belongs to two or more salary ranges, the concatenated list will be a mess. Thus, I create different dataframes for different salary ranges and then combine them as one.

In [3]:
df_1 = pd.DataFrame()
df_1['Link'] = job_urls['0-70000']
df_1['Salary Range'] = '0-70000'

In [4]:
df_2 = pd.DataFrame()
df_2['Link'] = job_urls['70000-120000']
df_2['Salary Range'] = '70000-120000'

In [5]:
df_3 = pd.DataFrame()
df_3['Link'] = job_urls['120000-999999']
df_3['Salary Range'] = '120000-999999'

In [6]:
url_pd = df_1.append(df_2).append(df_3)

In [7]:
url_pd['Link_1'] =['https://www.seek.com.au' + re.search('(\/job\/[0-9]{8})',i.attrs['href']).group(0) for i in url_pd['Link']]

In [8]:
url_pd['Title'] = [i.text for i in url_pd['Link']]

In [9]:
url_pd = url_pd.drop('Link',axis='columns')

In [10]:
columns = ['Salary Range', 'Link', "Title"]
url_pd.columns=columns

We can obtain a DataFrame which contains the urls for Data Analyst jobs for different salary ranges.

In [11]:
url_pd.head(10)

,Salary Range,Link,Title
0,0-70000,https://www.seek.com.au/job/39582436,Junior Market Research Data Analyst
1,0-70000,https://www.seek.com.au/job/39666286,Graduate / Junior GIS Analyst
2,0-70000,https://www.seek.com.au/job/39756451,Junior Data Analyst
3,0-70000,https://www.seek.com.au/job/39763876,Data Analyst
4,0-70000,https://www.seek.com.au/job/39755816,Data Analyst / Database Officer
5,0-70000,https://www.seek.com.au/job/39756108,Data & Billing Analyst - Commercial industry
6,0-70000,https://www.seek.com.au/job/39746760,Data Analyst
7,0-70000,https://www.seek.com.au/job/39753046,Data Analyst
8,0-70000,https://www.seek.com.au/job/39718063,Data Entry Officer/Junior Analyst
9,0-70000,https://www.seek.com.au/job/39764561,Investment Data Analyst (Graduate)


In [12]:
#for stemming
import re

def replace(string, substitutions):

    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

Start scrapying details for each url.

In [13]:
job_listing_date=[]
job_expiry_date=[]
job_title = []
job_teaser=[]
job_advertiser=[]
job_area=[]
job_worktype=[]
job_classification=[]
job_salary=[]
job_salary_type=[]
job_description_details=[]
job_description_strong = []
job_location =[]

for index, job in enumerate(url_pd['Link']):
    # iterate through all the urls
    r = requests.get(job)
    
    # turn into a BeautifulSoup object
    soup = BeautifulSoup(r.text, 'lxml')
    
    # find job descriptions
    jd_data = soup.find_all(attrs={'data-automation':'server-state'})[0]
    jd_raw= jd_data.text[3:].split('\n  ')[1][25:][:-1]
    jd_dict = json.loads(jd_raw)
    jd_dashboard = jd_dict['jobdetails']['result']
    
    if jd_dashboard['listingDate']:
        job_listing_date.append(jd_dashboard['listingDate'])
    else:
        job_listing_date.append('N/A')
        
    if jd_dashboard['expiryDate']:
        job_expiry_date.append(jd_dashboard['expiryDate'])
    else:
        job_expiry_date.append('N/A')
        
    if jd_dashboard['title']:
        job_title.append(jd_dashboard['title'])
    else:
        job_title.append('N/A')
        
    if jd_dashboard['teaser']:
        job_teaser.append(jd_dashboard['teaser'])
    else:
        job_teaser.append('N/A')
        
    if jd_dashboard['advertiser']['description']:
        job_advertiser.append(jd_dashboard['advertiser']['description'])
    else:
        job_advertiser.append('N/A')
        
    if jd_dashboard['locationHierarchy']['area']:
        job_area.append(jd_dashboard['locationHierarchy']['area'])
    else:
        job_area.append('N/A')
        
    if jd_dashboard['workType']:
        job_worktype.append(jd_dashboard['workType'])
    else:
        job_worktype.append('N/A')
        
    if jd_dashboard['classification']['description']:
        job_classification.append(jd_dashboard['classification']['description'])
    else:
        job_classification.append('N/A')
        
    if jd_dashboard['salary']:
        job_salary.append(jd_dashboard['salary'])
    else:
        job_salary.append('N/A')
        
    if jd_dashboard['salaryType']:
        job_salary_type.append(jd_dashboard['salaryType'])
    else:
        job_salary_type.append('N/A')
        
    if jd_dashboard['locationHierarchy']['city']:
        job_location.append(jd_dashboard['locationHierarchy']['city'])
    else:
        job_location.append('N/A')
        
    
    jd_detail = soup.find(attrs={'data-automation':'mobileTemplate'})
    job_description_details.append(jd_detail)
    

In [14]:
url_pd['Listing Date'] = job_listing_date
url_pd['Expiry Date'] =job_expiry_date
url_pd['Job Title']=job_title
url_pd['Job Teaser']=job_teaser
url_pd['Advertiser'] = job_advertiser
url_pd['Area'] = job_area
url_pd['Work Type'] = job_worktype
url_pd['Classification'] = job_classification
url_pd['Salary'] = job_salary
url_pd['Salary Type'] = job_salary_type
url_pd['JD'] = job_description_details
url_pd['Location'] =job_location


Check the type for 'JD' column and start cleaning.

In [15]:
type(job_description_details)

list

In [16]:
job_description_clean=[]
job_description_strong = []
for i in range(0,len(url_pd['JD'])):
    try:
        strong_word = job_description_details[i].find_all('strong')
        strong_word_list = [u.get_text(strip=True) for u in strong_word if strong_word != False]
        job_description_strong.append(strong_word_list)

        string = job_description_details[i].get_text()
        repla = {u'\xa0':'  ',u'\xe2\x80\x9d':'  ', u'\n':'  '}
        job_description_clean.append(replace(string,repla))
    except:
        job_description_strong.append('N/A')
        job_description_clean.append('N/A')

In [17]:
url_pd['Job Description'] = job_description_clean
url_pd['Strong Words'] = job_description_strong 

In [18]:
data_analyst_jobs = url_pd.copy()

In [19]:
data_analyst_jobs.to_csv('data_analyst_scraping.csv')

In [20]:
data_analyst_jobs.tail(10)

,Salary Range,Link,Title,Listing Date,Expiry Date,Job Title,Job Teaser,Advertiser,Area,Work Type,Classification,Salary,Salary Type,JD,Location,Job Description,Strong Words
974,120000-999999,https://www.seek.com.au/job/39719317,National Business Manager - Specialist AU,2019-08-16T01:55:47.000Z,2019-09-15T13:59:59.000Z,National Business Manager - Specialist AU,As the National Business Manager - Specialist ...,Vitaco Health NZ Ltd,Ryde & Macquarie Park,Full Time,Sales,N/A,AnnualPackage,[[[National Business Manager - Specialist AU- ...,Sydney,National Business Manager - Specialist AU- Syd...,[National Business Manager - Specialist AU- Sy...
975,120000-999999,https://www.seek.com.au/job/39661451,Project Manager,2019-08-08T05:48:58.000Z,2019-09-07T13:59:59.000Z,Project Manager,Bring your engineering project management expe...,Kinexus,N/A,Full Time,Government & Defence,N/A,AnnualPackage,"[[[About the client]], [ ], [Our client is a w...",ACT,About the client Our client is a well-known en...,"[About the client, About the role, Key Duties ..."
976,120000-999999,https://www.seek.com.au/job/39746608,Technical BA,2019-08-20T05:32:11.000Z,2019-09-19T13:59:59.000Z,Technical BA,Work on the Telephony Upgrade project working ...,Exclaim IT,N/A,Contract/Temp,Information & Communication Technology,N/A,HourlyRate,"[[[Duties and Responsibilities:]], [[Leading w...",ACT,Duties and Responsibilities:Leading workshops ...,"[Duties and Responsibilities:, Mandatory Skill..."
977,120000-999999,https://www.seek.com.au/job/39766902,Test Data & Automation Manager,2019-08-22T05:09:36.000Z,2019-09-21T05:09:36.000Z,Test Data & Automation Manager,Exciting opportunity for experienced Automatio...,face2face Recruitment,CBD & Inner Suburbs,Contract/Temp,Government & Defence,N/A,HourlyRate,"[[[Below is the Seek ad]], , [ , [face2face R...",Melbourne,Below is the Seek ad face2face Recruitment re...,"[Below is the Seek ad, About the Role:, Apply:]"
978,120000-999999,https://www.seek.com.au/job/39652914,Geotechnical Engineer,2019-08-07T06:56:35.000Z,2019-09-06T13:59:59.000Z,Geotechnical Engineer,An exciting opportunity has arisen for an expe...,Private Advertiser,N/A,Full Time,"Mining, Resources & Energy",$130k - $160k p.a. + Super,AnnualPackage,[[A well respected principal mining company of...,Dubbo & Central NSW,A well respected principal mining company offe...,"[The Mine Site, Role, Your Experience & Value,..."
979,120000-999999,https://www.seek.com.au/job/39715127,Business Analyst | Contract,2019-08-15T07:47:58.000Z,2019-09-14T13:59:59.000Z,Business Analyst | Contract,"Department of Education, are currently seeking...",Hudson,CBD & Inner Suburbs,Contract/Temp,Information & Communication Technology,Up to $695 p.d.,HourlyRate,"[[* Initial contract until December 2019, [], ...",Brisbane,* Initial contract until December 2019* Brisba...,[The successful candidate will be responsible ...
980,120000-999999,https://www.seek.com.au/job/39651265,Pega Lead BA,2019-08-07T05:15:07.000Z,2019-09-06T05:15:07.000Z,Pega Lead BA,Pega certified Business Architect with 7 + yea...,Virtusa Consulting & Services Pty Ltd,"CBD, Inner West & Eastern Suburbs",Full Time,Information & Communication Technology,"$130,000 - $179,999",AnnualPackage,"[[[JD - LBA]], , [ , [Lead the business team ...",Sydney,JD - LBA Lead the business team in gathering ...,"[JD - LBA, Requirements]"
981,120000-999999,https://www.seek.com.au/job/39652171,Azure Architect - Melbourne,2019-08-07T06:08:48.000Z,2019-09-06T06:08:48.000Z,Azure Architect - Melbourne,"We are hiring for our client, a leading Servic...",Ariston Partners Australia Pty Ltd,CBD & Inner Suburbs,Contract/Temp,Design & Architecture,N/A,HourlyRate,"[[We are hiring for our client, a leading Serv...",Melbourne,"We are hiring for our client, a leading Servic...","[1) Azure Architect, Responsibilities, 2), Azu..."
982,120000-999999,https://www.seek.com.au/job/39596212,CI Specialist,2019-07-31T04:53:20.000Z,2019-08-30T13:59:59.000Z,CI Specialist,Seeking an 